# APPROACH 2 Code :  Ensemble Model

### Step 1: Data Preparation

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.linear_model import Lasso, Ridge , LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor , VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score, GridSearchCV

##### 1.1 Load and prepare the dataset

In [4]:
data = pd.read_csv("data.csv")

In [5]:
# Replace infinite values with a large number (e.g., 1e10)
data['AI_Workload_Ratio'].replace([np.inf, -np.inf], 1e10, inplace=True)

In [6]:
missing_values = data.isnull().sum()
print(missing_values)

Job titles           0
AI Impact            0
Tasks                0
AI models            0
AI_Workload_Ratio    0
Domain               0
dtype: int64


In [7]:
infinite_values = data[data == np.inf].sum()
print(infinite_values)

Job titles             0
AI Impact              0
Tasks                0.0
AI models            0.0
AI_Workload_Ratio    0.0
Domain                 0
dtype: object


In [8]:
data['AI Impact'] = data['AI Impact'].str.rstrip('%').astype(float)/100

##### 1.2 Handle missing values (if any)

In [9]:
missing_values = data.isnull().sum()

inf_values = (data['AI_Workload_Ratio'] == float('inf')).sum()

missing_values, inf_values

(Job titles           0
 AI Impact            0
 Tasks                0
 AI models            0
 AI_Workload_Ratio    0
 Domain               0
 dtype: int64,
 0)

##### 1.3 Encode categorical variables


In [10]:
data = pd.get_dummies(data, columns=["Job titles", "Domain"], drop_first=True)

In [11]:
data.columns

Index(['AI Impact', 'Tasks', 'AI models', 'AI_Workload_Ratio',
       'Job titles_3D Animator', 'Job titles_3D Artist',
       'Job titles_3D Designer', 'Job titles_3D Modeler',
       'Job titles_911 Dispatcher', 'Job titles_911 Operator',
       ...
       'Job titles_Zoologist', 'Domain_Communication & PR',
       'Domain_Construction', 'Domain_Data & IT', 'Domain_Hospitality',
       'Domain_Law Enforcement', 'Domain_Leadership & Strategy',
       'Domain_Medical & Healthcare', 'Domain_Sales & Marketing',
       'Domain_Supply Chain & Logistics'],
      dtype='object', length=4718)

##### 1.4 Split the data into features (X) and target (y)

In [12]:
X = data.drop(columns=["AI Impact"])
y = data["AI Impact"]

##### 1.5 Split the data into training and testing sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### 1.6 Choose a regression model and train it

In [14]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Random Forest Regression", RandomForestRegressor()),
    ("Gradient Boosting Regression", GradientBoostingRegressor()),
    ("Lasso Regression", Lasso(alpha=0.01)),  
    ("Ridge Regression", Ridge(alpha=0.01)), 
]

##### 1.7 Training and evaluating multiple models 

In [15]:
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r_squared = r2_score(y_test, y_pred)
    
    print(f"Model: {name}")
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R-squared: {r_squared}')
    print("--------")

Model: Linear Regression
MSE: 0.039977242787945205
RMSE: 0.1999430988755181
R-squared: -0.10282736474255905
--------
Model: Random Forest Regression
MSE: 0.020803843736730363
RMSE: 0.14423537616247398
R-squared: 0.426097285238175
--------
Model: Gradient Boosting Regression
MSE: 0.02108342092463246
RMSE: 0.14520131171801604
R-squared: 0.4183847630161779
--------
Model: Lasso Regression
MSE: 0.029024849945861398
RMSE: 0.17036680998909792
R-squared: 0.1993094934627484
--------


C:\Users\Anubhav Chachra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Model: Ridge Regression
MSE: 0.029027314147090415
RMSE: 0.17037404188165053
R-squared: 0.1992415150740935
--------


In [16]:
param_grid_rf = {
    'n_estimators': [10,50, 100],  
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]
}
param_grid_gb = {
    'n_estimators': [10, 50, 100],  
    'learning_rate': [0.05, 0.1, 0.2],  
    'max_depth': [3, 4, 5]  
}

grid_search_rf = GridSearchCV(RandomForestRegressor(), param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_search_rf.fit(X_train, y_train)
grid_search_gb = GridSearchCV(GradientBoostingRegressor(), param_grid_gb, cv=5, scoring='neg_mean_squared_error')
grid_search_gb.fit(X_train, y_train)

rf_feature_selector = SelectFromModel(RandomForestRegressor(n_estimators=grid_search_rf.best_params_['n_estimators']))
rf_feature_selector.fit(X_train, y_train)
X_train_selected = rf_feature_selector.transform(X_train)
X_test_selected = rf_feature_selector.transform(X_test)

best_rf = grid_search_rf.best_estimator_
best_gb = grid_search_gb.best_estimator_

ensemble_model = VotingRegressor(estimators=[
    ('Random Forest', best_rf),
    ('Gradient Boosting', best_gb)
])

ensemble_model.fit(X_train_selected, y_train)
y_pred_ensemble = ensemble_model.predict(X_test_selected)

mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
rmse_ensemble = np.sqrt(mse_ensemble)
r_squared_ensemble = r2_score(y_test, y_pred_ensemble)

print("Ensemble Model Performance:")
print(f'MSE: {mse_ensemble}')
print(f'RMSE: {rmse_ensemble}')
print(f'R-squared: {r_squared_ensemble}')


Ensemble Model Performance:
MSE: 0.01894561221960177
RMSE: 0.1376430609206355
R-squared: 0.47735916385213595
